In [ ]:
%matplotlib inline

# Extract bike parking spaces from mongoDB, export as SVG for nesting, stitch nested SVGs 🚲 🚲 🚲

This notebook extracts geometries (areas, like polygons of parking spaces) from a mongoDB, then exports all areas in an svg file for nesting with SVGNest. One bin is used for parking spaces, one extra bin is used for spot parking spaces.

Created on:  2016-12-06  
Last update: 2017-04-13  
Contact: michael.szell@moovel.com, michael.szell@gmail.com (Michael Szell)

## Preliminaries

### Parameters

In [ ]:
cityname = "tokyo"

mode = "bike" # do bike here. car is another file
bikeparkw = 0.8
bikeparkh = 2

pathdatain = 'output/'+cityname+'/'+mode+'in/'
pathdataout = 'output/'+cityname+'/'+mode+'out/'

# manually excluding nodes that are tagged in OSM both as polygon and node
excludenodes = [1616515071, 1455723400]

### Imports

In [ ]:
from __future__ import unicode_literals
import sys
import csv
import os
import math
from random import shuffle, choice, uniform
import random
import pprint
pp = pprint.PrettyPrinter(indent=4)
from collections import defaultdict
import time
import datetime
import numpy as np
from numpy import *
from scipy import stats
import pyprind
import itertools
import logging
from ast import literal_eval as make_tuple
from collections import OrderedDict
from retrying import retry
from copy import deepcopy

import json
from xml.dom import minidom
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from shapely import affinity
from functools import partial
import pyproj
Projection = pyproj.Proj("+proj=merc +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs")
from scipy import spatial
from haversine import haversine
from scipy.ndimage.interpolation import rotate
from scipy.spatial import ConvexHull

import pymongo
from pymongo import MongoClient

# plotting stuff
import matplotlib.pyplot as plt

### DB Connection

In [ ]:
client = MongoClient()
db_derived = client[cityname+'_derived']
ways = db_derived['ways']
cursor = ways.find({"$and": [{"properties.amenity.amenity": "bicycle_parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 1 }}]}).sort("properties_derived.area",-1)
numparkingareas = cursor.count()
print("There are " + str(numparkingareas) + " " + mode + " parking spaces in " + cityname)

db_raw = client[cityname+'_raw']
nodes = db_raw['nodes']
cursornodes = nodes.find({"$and": [{"tags.amenity.amenity": "bicycle_parking"}, { "tags.capacity.capacity": { "$exists": True }}]})
numparkingspots = cursornodes.count()
print("There are " + str(numparkingspots) + " " + mode + " parking spots in " + cityname)


### Functions

In [ ]:
def coordinatesToSVGString(coo, coolatlon, xoffset = 0, yoffset = 0, idname = "", classname = "", rot = 0, centroidlatlon = [0,0]):
    svgstring = "\n  <polygon"
    if idname:
        svgstring += " id=\""+idname+"\""
    if classname:
        svgstring += " class=\""+classname+"\""
    svgstring += " points=\""
    strxylist = [str(coo[i][0]+xoffset)+","+str(coo[i][1]+yoffset) for i in range(coo.shape[0])]
    for s in strxylist:
        svgstring += s+" "
    svgstring += "\""
    svgstring += " moovel_rot=\""+str(rot)+"\"" # pseudo-namespace, because svgnest strips namespace info. http://stackoverflow.com/questions/15532371/do-svg-docs-support-custom-data-attributes
    centroid = [Polygon(coo).centroid.x, Polygon(coo).centroid.y]
    svgstring += " moovel_centroid=\""+str(centroid[0]+xoffset)+","+str(centroid[1]+yoffset)+"\""
    svgstring += " moovel_centroidlatlon=\""+str(centroidlatlon[0])+","+str(centroidlatlon[1])+"\""
    svgstring += " moovel_pointslatlon=\""
    strxylist = [str(coolatlon[i][0])+","+str(coolatlon[i][1]) for i in range(coolatlon.shape[0])]
    for s in strxylist:
        svgstring += s+" "
    svgstring += "\""
    svgstring += "/>"
    return svgstring

def drawPolygon(poly, title=""): # poly is a shapely Polygon
    x, y = poly.exterior.xy
    fig = plt.figure(figsize=(4,4), dpi=90)
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.plot(x, y)
    
def getLargestSubPolygon(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = 0
        j = 0
        for i in range(len(multipoly)):
            if multipoly[i].area > a:
                j = i
                a = multipoly[i].area
        return multipoly[j]
    
def getSmallestSubPolygon(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = float("inf")
        j = 0
        for i in range(len(multipoly)):
            if multipoly[i].area < a:
                j = i
                a = multipoly[i].area
        return multipoly[j]

def getTwoLargestSubPolygons(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give two largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = [multipoly[i].area for i in range(len(multipoly))]
        sortorder = sorted(range(len(a)), key=lambda k: a[k], reverse=True) # http://stackoverflow.com/questions/7851077/how-to-return-index-of-a-sorted-list
        return MultiPolygon([ multipoly[i] for i in sortorder[0:2] ])

def rotationToSmallestWidthRecursive(poly, maxdepth = 3, w = float("inf"), rot = 0, rotdelta = 10, depth = 1): # poly is a shapely polygon
    # unit: degrees
    # returns the angle the polygon needs to be rotated to be at minimum width
    # Note: Is not guaranteed to converge to the global minimum
    # Requires import numpy as np, from shapely import affinity
    if depth <= maxdepth:
        for theta in np.arange(rot-rotdelta*9, rot+rotdelta*9, rotdelta):
            temp = affinity.rotate(poly, theta, origin='centroid')
            x, y = temp.exterior.coords.xy
            temp = np.array([[x[i],y[i]] for i in range(len(x))])
            objectwidth = max(temp[:, 0])-min(temp[:, 0])
            if objectwidth < w:
                w = objectwidth
                rot = theta
        return rotationToSmallestWidthRecursive(poly, maxdepth, w, rot, rotdelta/10, depth+1)
    else:
        return rot
    
def getCoordinatesFromSVG(filepath, reversexdir = False, b = 1, xml = False): # The SVG needs to have polygons with ids, embedded in gs
    doc = minidom.parse(filepath)  # parseString also exists
    path_strings = [path.getAttribute('points') for path
                    in doc.getElementsByTagName('polygon')]
    pathlatlon_strings = [path.getAttribute('moovel_pointslatlon') for path
                    in doc.getElementsByTagName('polygon')]
    id_strings = [path.getAttribute('id') for path
                    in doc.getElementsByTagName('polygon')]
    class_strings = [path.getAttribute('class') for path
                    in doc.getElementsByTagName('polygon')]
    if not xml:
        g_strings = [path.getAttribute('transform') for path
                    in doc.getElementsByTagName('g')]
    rot_strings = [path.getAttribute('moovel_rot') for path
                    in doc.getElementsByTagName('polygon')]
    centroidlatlon_strings = [path.getAttribute('moovel_centroidlatlon') for path
                    in doc.getElementsByTagName('polygon')]
    doc.unlink()
    data = dict()
    numbins = 0
    for i,(path,pathlatlon) in enumerate(zip(path_strings, pathlatlon_strings)):
        if class_strings[i] == "bin":
            numbins += 1
        if numbins == b:
            path = path.split()
            coo = []
            for temp in path:
                p = temp.split(",")
                if xml:
                    trans = [0,0]
                else:
                    try:
                        trans = g_strings[i] # looks like this: "translate(484.1119359029915 -1573.8819930603422) rotate(0)"
                        trans = trans.split()
                        trans = [float(trans[0][10:]), float(trans[1][0:-1])] # gives [484.1119359029915,-1573.8819930603422]
                    except:
                        trans = [0,0]
                if reversexdir:
                    coo.append([-(float(p[0])+trans[0]), float(p[1])+trans[1]])
                else:
                    coo.append([float(p[0])+trans[0], float(p[1])+trans[1]])
            pathlatlon = pathlatlon.split()
            coolatlon = []
            for temp in pathlatlon:
                p = temp.split(",")
                coolatlon.append([float(p[0]), float(p[1])])
            data[id_strings[i]] = dict()
            data[id_strings[i]]["coordinates"] = coo
            data[id_strings[i]]["coordinateslatlon"] = coolatlon
            data[id_strings[i]]["rot"] = rot_strings[i]
            data[id_strings[i]]["class"] = class_strings[i]
            data[id_strings[i]]["centroidlatlon"] = centroidlatlon_strings[i].split(",")
        elif numbins > b:
            break
    return data

## Get parking spaces for multiple SVG bins

In [ ]:
cursor = ways.find({"$and": [{"properties.amenity.amenity": "bicycle_parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 1 }}]}).sort("properties_derived.area",-1)
cursornodes = nodes.find({"$and": [{"tags.amenity.amenity": "bicycle_parking"}, { "tags.capacity.capacity": { "$exists": True }}]})

random.seed(1)
scale = 0.6
erectparts = True
randomrotateparts = False
smallvsmedium = 11
buffereps = 5 # should be the same number as the distances between parts in SVGNest
height = 1200
width = 600-2*buffereps
draw = False # for debugging purposes (drawing all big parts and bins) set this to True
eps = 0.000001

# pre-select all parts
idsused = set()
idsnotused = set()
idsusednodes = set()
idsnotusednodes = set()
alltiles = []
alltileskeys = []
alltilesarea = 0
for i,way in enumerate(cursor):
    npway = np.asarray(way["geometry"]["coordinates"])
    centroidlatlon = [Polygon(npway).centroid.x, Polygon(npway).centroid.y]
    npwayxy = [Projection(npway[i][0], npway[i][1]) for i in range(npway.shape[0])]
    npwayxy = np.asarray([[npwayxy[i][0],-npwayxy[i][1]] for i in range(npway.shape[0])])
    if erectparts:
        rot = 90+rotationToSmallestWidthRecursive(Polygon(npwayxy))
    elif randomrotateparts:
        rot = uniform(10, 350)
    else:
        rot = 0
    if rot:
        temp = affinity.rotate(Polygon(npwayxy), rot, origin='centroid', use_radians=False)
        x, y = temp.exterior.coords.xy
        npwayxy = np.array([[x[i],y[i]] for i in range(len(x))])
    objectwidth = max(npwayxy[:, 0])-min(npwayxy[:, 0])
    npwayxy[:, 0] -= min(npwayxy[:, 0])
    npwayxy[:, 1] -= min(npwayxy[:, 1])
    npwayxy *= scale
    objectwidth *= scale
    if objectwidth < width:
        objectheight = max(npwayxy[:, 1])
        idsnotused.add(int(way["_id"]))
        coo = [[npwayxy[k][0], npwayxy[k][1]] for k in range(npwayxy.shape[0])]
        coolatlon = [[npway[k][0], npway[k][1]] for k in range(npway.shape[0])]
        area = Polygon(coo).buffer(buffereps/2).area
        alltiles.append( { "_id": int(way["_id"]), "width": objectwidth, "height": objectheight, "area": area, "coordinates": coo , "coordinateslatlon": coolatlon, "rot": rot, "centroidlatlon": centroidlatlon})
        alltileskeys.append(int(way["_id"]))
        alltilesarea += area
    else:
        print("Object "+str(way["_id"])+" was too wide (" +str(objectwidth)+ " pixel) and was ignored.")
        
# Generation of polygons from point parking
capacitiesall = []
alltilesareanodes = 0
for i,node in enumerate(cursornodes):
    try: # sometimes capacity is not an integer
        capacity = int(node["tags"]["capacity"]["capacity"])
    except:
        capacity = 0
    if capacity and node["_id"] not in excludenodes:
        centroidlatlon = node["loc"]["coordinates"]
        if capacity <= 20:
            xd = capacity*bikeparkw/2
            yd = bikeparkh/2
        else:
            xd = math.sqrt(capacity)*bikeparkh/2
            yd = math.sqrt(capacity)*bikeparkw/2
        npwayxy = [[-xd, -yd], [xd, -yd], [xd, yd], [-xd, yd]]
        npwayxy = np.asarray([[npwayxy[i][0],-npwayxy[i][1]] for i in range(4)])
        objectwidth = max(npwayxy[:, 0])-min(npwayxy[:, 0])
        npwayxy[:, 0] -= min(npwayxy[:, 0])
        npwayxy[:, 1] -= min(npwayxy[:, 1])
        npwayxy *= scale
        objectwidth *= scale
        if objectwidth < width:
            objectheight = max(npwayxy[:, 1])
            idsnotusednodes.add(int(node["_id"]))
            coo = [[npwayxy[k][0], npwayxy[k][1]] for k in range(npwayxy.shape[0])]
            coolatlon = [centroidlatlon, centroidlatlon, centroidlatlon, centroidlatlon]
            area = Polygon(coo).buffer(buffereps/2).area
            alltiles.append( { "_id": int(node["_id"]), "width": objectwidth, "height": objectheight, "area": area, "coordinates": coo , "coordinateslatlon": coolatlon, "rot": rot, "centroidlatlon": centroidlatlon})
            alltileskeys.append(int(node["_id"]))
            alltilesareanodes += area
            capacitiesall.append(capacity)
        else:
            print("Object "+str(node["_id"])+" was too wide (" +str(objectwidth)+ " pixel) and was ignored.")
sortind = [i[0] for i in sorted(enumerate(capacitiesall), key=lambda x:x[1], reverse=True)]

# Parking spaces
bigbin = Polygon([[0,0], [width,0], [width, height], [0, height]])
bigbinarea = bigbin.area
# change the big bin area according to the leftover tiles area
heightbigbin = 1.28 * height * alltilesarea/bigbinarea
bigbin = Polygon([[0,0], [width,0], [width, heightbigbin], [0, heightbigbin]])     
# Fill with parts
binbound = np.array(bigbin.exterior.coords)
xpos = 0
ypos = heightbigbin+1 # start placing the elements below the bin
yextent = 0
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(heightbigbin)+"px\">"
svg += coordinatesToSVGString(binbound, binbound, 0, 0, str(0), "bin")
cnt = 0
for j in range(len(idsnotused)):
    if len(idsnotused) == 0:
        break
    tile = alltiles[j]
    if tile["width"] <= width:
        if xpos + tile["width"] + 1 <= width: # there is space in this row
            xdelta = (xpos+1)
            ydelta = ypos
        else: # new row
            xdelta = 0
            ypos += yextent
            yextent = 0
            ydelta = ypos
            xpos = 0
        svg += coordinatesToSVGString(np.array([[tile["coordinates"][k][0], tile["coordinates"][k][1]] for k in range(np.array(tile["coordinates"]).shape[0])]), np.array([[tile["coordinateslatlon"][k][0], tile["coordinateslatlon"][k][1]] for k in range(np.array(tile["coordinateslatlon"]).shape[0])]), xdelta, ydelta, str(tile["_id"]), "tile", tile["rot"], tile["centroidlatlon"])
        yextent = max([yextent, tile["height"]])
        xpos += tile["width"]+1
        idsused.add(tile["_id"])
        idsnotused.remove(tile["_id"])
        cnt += 1
    else:
        print("Object "+str(way["_id"])+" was too wide (" +str(max(npwayxy[:, 0]))+ " pixel) and could not be placed.")
svg += "\n</svg>"
with open(pathdatain + cityname + mode + "parking"+ str(0).zfill(3)  +"in.svg", "w") as f:
    f.write(svg)
    

# Parking spots
extrabin = Polygon([[0,0], [width,0], [width, height], [0, height]])
extrabinarea = extrabin.area
# change the big bin area according to the leftover tiles area
heightextrabin = 1.25 * height * alltilesareanodes/extrabinarea
extrabin = Polygon([[0,0], [width,0], [width, heightextrabin], [0, heightextrabin]])     
# Fill with parts
binbound = np.array(extrabin.exterior.coords)
xpos = 0
ypos = 0
yextent = 0
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(heightextrabin)+"px\">"
for j in sortind:
    if len(idsnotusednodes) == 0:
        break
    tile = alltiles[j+cnt]
    if tile["width"] <= width:
        if xpos + tile["width"] + buffereps <= width: # there is space in this row
            xdelta = (xpos+1)
            ydelta = ypos
        else: # new row
            xdelta = 0
            ypos += yextent + buffereps
            yextent = 0
            ydelta = ypos
            xpos = 0
        svg += coordinatesToSVGString(np.array([[tile["coordinates"][k][0], tile["coordinates"][k][1]] for k in range(np.array(tile["coordinates"]).shape[0])]), np.array([[tile["coordinateslatlon"][k][0], tile["coordinateslatlon"][k][1]] for k in range(np.array(tile["coordinateslatlon"]).shape[0])]), xdelta, ydelta, str(tile["_id"]), "tile", tile["rot"], tile["centroidlatlon"])
        yextent = max([yextent, tile["height"]])
        xpos += tile["width"]+buffereps
        idsusednodes.add(tile["_id"])
        idsnotusednodes.remove(tile["_id"])
    else:
        print("Object "+str(way["_id"])+" was too wide (" +str(max(npwayxy[:, 0]))+ " pixel) and could not be placed.")
svg += "\n</svg>"
with open(pathdataout + str(1).zfill(3)  +".svg", "w") as f:
    f.write(svg)
                
print("First export done. " + str(len(idsnotused)+len(idsnotusednodes))+" tiles were not used.")

The result is a file in {{pathdataout}}. Use SVGNest on this file. Move the file returned from SVGNest into {{pathdataout}}. There, also a nother file called 001.svg was generated. These two files are stitched together next.

## After SVGNest was executed, stitch together the two parts

In [ ]:
# read SVG
alltilesfinal = []
tiles = getCoordinatesFromSVG(pathdataout + str(0).zfill(3) + ".svg")
maxy = 0
for key in tiles:
    if tiles[key]["class"] == "tile":
        npwayxy = np.array(tiles[key]["coordinates"])
        npway = np.array(tiles[key]["coordinateslatlon"])
        maxy = max(np.append(npwayxy[:,1], maxy))
        npwayxy = [[npwayxy[k,0], npwayxy[k,1]] for k in range(npwayxy.shape[0])]
        alltilesfinal.append({key: {"coordinates": npwayxy, "coordinateslatlon": npway, "rot": tiles[key]["rot"], "centroidlatlon": tiles[key]["centroidlatlon"]}})
tiles = getCoordinatesFromSVG(pathdataout + str(1).zfill(3) + ".svg", False, 0, True)
maxy2 = 0
for key in tiles:
    if tiles[key]["class"] == "tile":
        npwayxy = np.array(tiles[key]["coordinates"])
        npway = np.array(tiles[key]["coordinateslatlon"])
        maxy2 = max(np.append(npwayxy[:,1], maxy2))
        npwayxy = [[npwayxy[k,0], npwayxy[k,1]+maxy+buffereps] for k in range(npwayxy.shape[0])]
        alltilesfinal.append({key: {"coordinates": npwayxy, "coordinateslatlon": npway, "rot": tiles[key]["rot"], "centroidlatlon": tiles[key]["centroidlatlon"]}})

# Export
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(maxy+maxy2+buffereps)+"px\">"
for j, tile in enumerate(alltilesfinal):
    for i in tile:
        svg += coordinatesToSVGString(np.array([[tile[i]["coordinates"][k][0], tile[i]["coordinates"][k][1]] for k in range(np.array(tile[i]["coordinates"]).shape[0])]), np.array([[tile[i]["coordinateslatlon"][k][0], tile[i]["coordinateslatlon"][k][1]] for k in range(np.array(tile[i]["coordinateslatlon"]).shape[0])]), 0, 0, i, "", tile[i]["rot"], tile[i]["centroidlatlon"])
svg += "\n</svg>"
with open(pathdataout + "all.svg", "w") as f:
    f.write(svg)